In [33]:
%load_ext autoreload
%autoreload 2
from adversarial_models.pgd import PGD
from models.discriminator import CLIPSVMDiscriminator
import torchvision
from torchvision import transforms
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

attack = PGD(model, eps=8 / 255, alpha=2 / 255, steps=10, random_start=True)
IMAGE_PATH = "images/dalle/dalle_148.jpg"

# Custom forward method for the attack to handle transformers models
def custom_forward(self, inputs, x):
    with torch.no_grad():
        outputs = model.vision_model(inputs.pixel_values)
        image_features = outputs.last_hidden_state[:, 0, :]
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        return image_features.squeeze().cpu().numpy()

# Assign the custom forward method
attack._targeted = False
attack.forward = custom_forward.__get__(attack)

In [70]:
transform = transforms.Compose([
    transforms.ToTensor(),  # This scales pixels to [0, 1]
])

In [71]:
# Load the image to torch
image = Image.open(IMAGE_PATH)
image = image.convert("RGB")
inputs = processor(images=image, return_tensors="pt")
inputs = inputs.to(model.device)
label = torch.tensor([0]).unsqueeze(0).to(model.device)

In [74]:
# Generate adversarial example
adversarial_image = attack(inputs, torch.tensor([0]))